In [ ]:
!pip install torch torchvision torchaudio
!pip install pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Download the dataset
!wget https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv


--2025-02-10 18:22:11--  https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 477907 (467K) [text/plain]
Saving to: ‘sms.tsv’

sms.tsv             100%[===================>] 466.71K  --.-KB/s    in 0.05s   

2025-02-10 18:22:12 (9.58 MB/s) - ‘sms.tsv’ saved [477907/477907]



In [ ]:
import pandas as pd
df = pd.read_csv('sms.tsv', sep='\t', header=None, names=['label','message'])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df['label']= df.label.map({'ham':0, 'spam':1})
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(df['message'], df['label'],test_size=0.2,random_state=42)
print(f"Training Samples:{len(X_train)}" )
print(f"Testing Samples:{len(X_test)}")

Training Samples:4457
Testing Samples:1115


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vect= vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [ ]:
import torch

X_train_tensor = torch.tensor(X_train_vect.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_vect.toarray(), dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

In [ ]:
import torch.nn as nn
class SpamClassifier(nn.Module):
  def __init__(self,input_dim):
    super(SpamClassifier, self).__init__()
    self.fc1=nn.Linear(input_dim,64)
    self.fc2=nn.Linear(64,1)
    #self.relu=nn.ReLU()
    self.sigmoid=nn.Sigmoid()
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.sigmoid(x)
    return x
input_dim = X_train_tensor.shape[1]
model = SpamClassifier(input_dim)
print(model)


SpamClassifier(
  (fc1): Linear(in_features=7702, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [21]:
criterion = nn.BCELoss()
optimizer =  torch.optim.Adam(model.parameters(),lr=0.01)

In [22]:
epochs = 100
for epoch in range(epochs):
  model.train()
  outputs=model(X_train_tensor)
  loss = criterion(outputs,y_train_tensor)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  print(f"Epoch[{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch[1/100], Loss: 0.1263
Epoch[2/100], Loss: 0.0960
Epoch[3/100], Loss: 0.0716
Epoch[4/100], Loss: 0.0536
Epoch[5/100], Loss: 0.0399
Epoch[6/100], Loss: 0.0298
Epoch[7/100], Loss: 0.0224
Epoch[8/100], Loss: 0.0170
Epoch[9/100], Loss: 0.0130
Epoch[10/100], Loss: 0.0101
Epoch[11/100], Loss: 0.0080
Epoch[12/100], Loss: 0.0064
Epoch[13/100], Loss: 0.0051
Epoch[14/100], Loss: 0.0042
Epoch[15/100], Loss: 0.0034
Epoch[16/100], Loss: 0.0028
Epoch[17/100], Loss: 0.0024
Epoch[18/100], Loss: 0.0020
Epoch[19/100], Loss: 0.0017
Epoch[20/100], Loss: 0.0015
Epoch[21/100], Loss: 0.0013
Epoch[22/100], Loss: 0.0011
Epoch[23/100], Loss: 0.0010
Epoch[24/100], Loss: 0.0009
Epoch[25/100], Loss: 0.0008
Epoch[26/100], Loss: 0.0007
Epoch[27/100], Loss: 0.0006
Epoch[28/100], Loss: 0.0006
Epoch[29/100], Loss: 0.0006
Epoch[30/100], Loss: 0.0005
Epoch[31/100], Loss: 0.0005
Epoch[32/100], Loss: 0.0005
Epoch[33/100], Loss: 0.0004
Epoch[34/100], Loss: 0.0004
Epoch[35/100], Loss: 0.0004
Epoch[36/100], Loss: 0.0004
E

In [23]:
model.eval()
with torch.no_grad():
  test_outputs = model(X_test_tensor)
  predicted = (test_outputs > 0.5).float()

In [24]:
accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 98.83%


In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test_tensor.numpy(), predicted.numpy(), target_names=['Ham', 'Spam']))


              precision    recall  f1-score   support

         Ham       0.99      1.00      0.99       966
        Spam       1.00      0.91      0.95       149

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [26]:
torch.save(model.state_dict(), 'spam_classifier.pth')


In [30]:
from google.colab import files
files.download('spam_classifier.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>